In [145]:
#Imports and API Key
#building in offsets

#Imports and API Key

import pandas as pd
import quandl
from scipy import stats
import scipy
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

from sklearn.pipeline import Pipeline

from basis_expansions.basis_expansions import (
    Polynomial, LinearSpline)

from regression_tools.dftransformers import (
    ColumnSelector, Identity, FeatureUnion, MapFeature, Intercept)

from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeClassifier

import xgboost as xgb

%aimport dataclean

quandl.ApiConfig.api_key = 'm8FYMyoCaJSbTrBASNHh'

In [146]:
#pulling data from Quandl

data = pd.read_csv('data_for_pull.csv') #staging the QUANDL keys to pull in CSV
cols = list(data['Var_name'].astype('str'))
dataset = quandl.get([val for val in data['Quandl Key']]) #looping through the QUANDL keys to pull it into one DF
dataset.columns = cols

In [147]:
dataset = dataset.drop(['PPI','CPI','EXPORTS', 'IMPORTS', 'TRADE_BALANCE', 'US_M2','HOUS_PERMS'], axis=1)

In [148]:
dataset = dataset.drop(['UNR'],axis=1)

In [149]:
unr = pd.read_csv('UNR_og_publish_formatted.csv')

In [150]:
unr = unr.drop(['Unnamed: 0','Year','Month'],axis=1)

In [151]:
unr['Date'] = pd.to_datetime(unr['Date'])

In [152]:
unr['Date'] = unr['Date'].apply(lambda x: x.strftime('%Y-%m'))

In [153]:
unr = unr.set_index('Date')

In [154]:
#dataset = dataset.join(unr, how='outer')

In [155]:
dataset.head()

,PMI,YUNR,GDP,CONS_SENT,HOME_SALES,PART_TIME,CPPR_PRICE,HOUS_STARTS,CAP_UTIL,PERS_SAVINGS,INT_RATE,US_NHOME_SALES,3YRT
1913-01-01,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1913-02-01,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1913-03-01,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1913-04-01,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1913-05-01,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [156]:
#pulling FED Yield Curve data

yields = pd.read_csv('Fed10Y_3M.csv')
yields['Date'] = pd.to_datetime(yields['Date'])
yields['Date'] = yields['Date'].apply(lambda x: x.strftime('%Y-%m'))
yields = yields.set_index('Date')
yields = yields.drop(['3 Month Treasury Yield', 'Rec_prob', 'NBER_Rec','Unnamed: 7'], axis=1) 

In [157]:
#need to write in special logic to factor in for 3YRT being a daily

treas = dataset['3YRT']
dataset = dataset.drop(columns = ['3YRT'])

In [158]:
treas = treas.resample('MS').mean()

In [159]:
dataset.head()

,PMI,YUNR,GDP,CONS_SENT,HOME_SALES,PART_TIME,CPPR_PRICE,HOUS_STARTS,CAP_UTIL,PERS_SAVINGS,INT_RATE,US_NHOME_SALES
1913-01-01,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1913-02-01,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1913-03-01,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1913-04-01,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1913-05-01,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [160]:
## back to working on the general data
dataset.index = dataset.index.strftime('%Y-%m') #converting the datetime index to Y/M so it is collapsable
dataset = dataset.groupby(dataset.index, as_index=True).agg(sum) #collapsing by Y/M

In [161]:
dataset.head()

,PMI,YUNR,GDP,CONS_SENT,HOME_SALES,PART_TIME,CPPR_PRICE,HOUS_STARTS,CAP_UTIL,PERS_SAVINGS,INT_RATE,US_NHOME_SALES
1913-01,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1913-02,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1913-03,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1913-04,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1913-05,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [162]:
#readd 3YRT back into data

dataset = dataset.join(treas, how='outer')
dataset = dataset.join(unr, how='outer')

In [163]:
#converting GDP quarterly data into monthly - need to convert it so it fills in the following 3 months

dataset = dataclean.convert_q_to_m(dataset, 'GDP')

#converting consumer sentiment into monthly

dataset = dataclean.convert_q_to_m(dataset, 'CONS_SENT')

/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [164]:
dataset.head()

,PMI,YUNR,GDP,CONS_SENT,HOME_SALES,PART_TIME,CPPR_PRICE,HOUS_STARTS,CAP_UTIL,PERS_SAVINGS,INT_RATE,US_NHOME_SALES,3YRT,UNR
1913-01-01,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,nan,NaN
1913-02-01,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,nan,NaN
1913-03-01,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,nan,NaN
1913-04-01,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,nan,NaN
1913-05-01,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,nan,NaN


In [165]:
offset_dict = {'PMI':1, 'UNR':1, 'YUNR':1,'HOME_SALES':1, 'PART_TIME':1, 'CPPR_PRICE':1, 'HOUS_STARTS':1, 'CAP_UTIL':1, 'PERS_SAVINGS':3, 'US_NHOME_SALES':1} #  'HOUS_PERMS':1,'US_M2':1,'EXPORTS':3, 'IMPORTS':3, 'TRADE_BALANCE':3,,'PPI':1,'CPI':1} #positives shift down, negatives shift up. Goal of this dict is to align data to when it gets released

In [166]:
for entry in offset_dict:
    #print(entry)
    #print(offset_dict[entry])
    dataset[str(entry)] = dataset[str(entry)].shift(offset_dict[entry])

In [167]:
#calculating change in GDP and converting Y into categorical values 
dataset['Recession'] = ((dataset['GDP'] - dataset['GDP'].shift(3)) < 0).astype(int)
dataset = dataset.drop(columns = ['GDP']) #dropping calc column and recession column from dataset, experimenting with taking out fed funds rate

In [168]:
#merge fed interest rate data here
dataset = dataset.join(yields, how='outer')

In [169]:
dataset.shape

(1286, 17)

In [170]:
dataset = dataset[552:]

In [171]:
dataset = dataset[:-12]

In [172]:
#substituting mean value in for missing values and adding dummy column to indicate where done.

#this could be leakage, use backwards looking means

for col in dataset.columns:
    if str(col)=='Recession':
        continue
    dataclean.clean_zeros(col, dataset)

In [173]:
dataset['3YRT'] = dataset['3YRT'].fillna(dataset['3YRT'].mean())

In [174]:
dataset.head()

,PMI,YUNR,CONS_SENT,HOME_SALES,PART_TIME,CPPR_PRICE,HOUS_STARTS,CAP_UTIL,PERS_SAVINGS,INT_RATE,...,Recession,10 Year Treasury Yield,3 Month Treasury Yield (Bond Equivalent Basis),Spread,HOME_SALES_PXY,CPPR_PRICE_PXY,HOUS_STARTS_PXY,CAP_UTIL_PXY,PERS_SAVINGS_PXY,US_NHOME_SALES_PXY
1959-01-01,60.500,12.100,90.800,3353.324,1081.000,165.089,1429.532,69.480,8.779,2.480,...,0.000,4.020,2.880,1.140,1,1,1,1,1,1
1959-02-01,64.400,11.600,90.800,3353.324,1022.000,165.089,1657.000,69.480,8.779,2.430,...,0.000,3.960,2.760,1.200,1,1,0,1,1,1
1959-03-01,66.900,11.100,90.800,3353.324,973.000,165.089,1667.000,69.480,8.779,2.800,...,0.000,3.990,2.860,1.130,1,1,0,1,1,1
1959-04-01,67.100,11.100,90.800,3353.324,1102.000,165.089,1620.000,69.480,11.300,2.960,...,0.000,4.120,3.010,1.110,1,1,0,1,0,1
1959-05-01,66.900,10.800,95.300,3353.324,1086.000,165.089,1590.000,69.480,10.600,2.900,...,0.000,4.310,2.900,1.410,1,1,0,1,0,1


In [175]:
dataset['UNR'] = dataset['UNR'].astype(float)

In [176]:
#adding momentum factors

momentum_cols = list(dataset.columns[:-6])

#momentum_cols.remove('PPI') #removing PPI and CPI because they need a different transformation
#momentum_cols.remove('CPI')
momentum_cols.remove('Recession')

for i in [1,3,12]:
    for col in momentum_cols:
        if 'PXY' in str(col): #adding logic so it doesn't create a momentum column out of PXY columns
            continue
        dataclean.create_momentum(col,dataset,i)

In [177]:
dataset.head()

,PMI,YUNR,CONS_SENT,HOME_SALES,PART_TIME,CPPR_PRICE,HOUS_STARTS,CAP_UTIL,PERS_SAVINGS,INT_RATE,...,HOUS_STARTS_12m_shift,CAP_UTIL_12m_shift,PERS_SAVINGS_12m_shift,INT_RATE_12m_shift,US_NHOME_SALES_12m_shift,3YRT_12m_shift,UNR_12m_shift,10 Year Treasury Yield_12m_shift,3 Month Treasury Yield (Bond Equivalent Basis)_12m_shift,Spread_12m_shift
1959-01-01,60.500,12.100,90.800,3353.324,1081.000,165.089,1429.532,69.480,8.779,2.480,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1959-02-01,64.400,11.600,90.800,3353.324,1022.000,165.089,1657.000,69.480,8.779,2.430,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1959-03-01,66.900,11.100,90.800,3353.324,973.000,165.089,1667.000,69.480,8.779,2.800,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1959-04-01,67.100,11.100,90.800,3353.324,1102.000,165.089,1620.000,69.480,11.300,2.960,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1959-05-01,66.900,10.800,95.300,3353.324,1086.000,165.089,1590.000,69.480,10.600,2.900,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [178]:
#CPI Calcs

#for i in [1,3,12]:
#    for col in ['CPI','PPI']:
#        dataclean.infl_momentum(col,dataset,i)

In [179]:
dataset.shape

(722, 71)

In [180]:
#cutoff most of missing data, Post March 2019, Prior 1959. CPI/PPI missing 2016 onward so need to cut that off
#dataset = dataset.iloc[552:]
#dataset = dataset.iloc[:-2]

#y = y.iloc[552:]
y = dataset['Recession']
dataset = dataset.drop(columns = ['Recession'])
X = dataset

In [181]:
### Data Prep Finished Here ###

In [182]:
y_shift = y.shift(0) #needs to be negative to look forward
y_shift = y_shift.fillna(0)

In [183]:
X_train = X.iloc[12:550]
X_test = X.iloc[550:]
y_train = y_shift.iloc[12:550]
y_test = y_shift.iloc[550:]

In [197]:
y_test

2004-11-01   0.000
2004-12-01   0.000
2005-01-01   0.000
2005-02-01   0.000
2005-03-01   0.000
2005-04-01   0.000
2005-05-01   0.000
2005-06-01   0.000
2005-07-01   0.000
2005-08-01   0.000
2005-09-01   0.000
2005-10-01   0.000
2005-11-01   0.000
2005-12-01   0.000
2006-01-01   0.000
2006-02-01   0.000
2006-03-01   0.000
2006-04-01   0.000
2006-05-01   0.000
2006-06-01   0.000
2006-07-01   0.000
2006-08-01   0.000
2006-09-01   0.000
2006-10-01   0.000
2006-11-01   0.000
2006-12-01   0.000
2007-01-01   0.000
2007-02-01   0.000
2007-03-01   0.000
2007-04-01   0.000
              ... 
2016-09-01   0.000
2016-10-01   0.000
2016-11-01   0.000
2016-12-01   0.000
2017-01-01   0.000
2017-02-01   0.000
2017-03-01   0.000
2017-04-01   0.000
2017-05-01   0.000
2017-06-01   0.000
2017-07-01   0.000
2017-08-01   0.000
2017-09-01   0.000
2017-10-01   0.000
2017-11-01   0.000
2017-12-01   0.000
2018-01-01   0.000
2018-02-01   0.000
2018-03-01   0.000
2018-04-01   0.000
2018-05-01   0.000
2018-06-01  

In [184]:
model = xgb.XGBClassifier(learning_rate=0.005,
                                   n_estimators=1000, #bump this and learning rate to make more trees, trees(not exact) = n*learning rate
                                   min_samples_leaf=4,
                                   max_depth=1,
                                   subsample=0.5)

model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.005, max_delta_step=0,
       max_depth=1, min_child_weight=1, min_samples_leaf=4, missing=None,
       n_estimators=1000, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=0.5)

In [185]:
probs = model.predict_proba(X_test)

In [186]:
log_loss(y_test, probs)

0.1482598616005203

In [187]:
roc_auc_score(y_test.values, probs[:,1:])

0.9895833333333334

In [188]:
results = pd.DataFrame(probs)
results['actual'] = y_test.values
results.index = y_test.index
pd.set_option('display.float_format', lambda x: '%.3f' % x)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(results)

               0     1  actual
2004-11-01 0.988 0.012   0.000
2004-12-01 0.984 0.016   0.000
2005-01-01 0.986 0.014   0.000
2005-02-01 0.986 0.014   0.000
2005-03-01 0.988 0.012   0.000
2005-04-01 0.976 0.024   0.000
2005-05-01 0.983 0.017   0.000
2005-06-01 0.986 0.014   0.000
2005-07-01 0.987 0.013   0.000
2005-08-01 0.984 0.016   0.000
2005-09-01 0.973 0.027   0.000
2005-10-01 0.981 0.019   0.000
2005-11-01 0.988 0.012   0.000
2005-12-01 0.990 0.010   0.000
2006-01-01 0.989 0.011   0.000
2006-02-01 0.987 0.013   0.000
2006-03-01 0.990 0.010   0.000
2006-04-01 0.989 0.011   0.000
2006-05-01 0.981 0.019   0.000
2006-06-01 0.989 0.011   0.000
2006-07-01 0.988 0.012   0.000
2006-08-01 0.987 0.013   0.000
2006-09-01 0.987 0.013   0.000
2006-10-01 0.988 0.012   0.000
2006-11-01 0.987 0.013   0.000
2006-12-01 0.988 0.012   0.000
2007-01-01 0.988 0.012   0.000
2007-02-01 0.985 0.015   0.000
2007-03-01 0.988 0.012   0.000
2007-04-01 0.985 0.015   0.000
2007-05-01 0.989 0.011   0.000
2007-06-

In [189]:
X_train

,PMI,YUNR,CONS_SENT,HOME_SALES,PART_TIME,CPPR_PRICE,HOUS_STARTS,CAP_UTIL,PERS_SAVINGS,INT_RATE,...,HOUS_STARTS_12m_shift,CAP_UTIL_12m_shift,PERS_SAVINGS_12m_shift,INT_RATE_12m_shift,US_NHOME_SALES_12m_shift,3YRT_12m_shift,UNR_12m_shift,10 Year Treasury Yield_12m_shift,3 Month Treasury Yield (Bond Equivalent Basis)_12m_shift,Spread_12m_shift
1960-01-01,58.200,11.100,93.800,3353.324,1000.000,165.089,1601.000,69.480,9.400,3.990,...,171.468,0.000,0.621,1.510,0.000,0.000,-0.900,0.700,1.580,-0.880
1960-02-01,61.500,10.900,93.800,3353.324,1015.000,165.089,1460.000,69.480,10.100,3.970,...,-197.000,0.000,1.321,1.540,0.000,0.000,-0.800,0.530,1.300,-0.770
1960-03-01,52.300,10.200,93.800,3353.324,1062.000,165.089,1503.000,69.480,11.000,3.840,...,-164.000,0.000,2.221,1.040,0.000,0.000,-1.300,0.260,0.520,-0.260
1960-04-01,47.800,11.500,93.800,3353.324,888.000,165.089,1109.000,69.480,10.900,3.920,...,-511.000,0.000,-0.400,0.960,0.000,0.000,-0.400,0.160,0.290,-0.130
1960-05-01,45.300,10.900,93.300,3353.324,1041.000,165.089,1289.000,69.480,10.600,3.850,...,-301.000,0.000,0.000,0.950,0.000,0.000,-0.300,0.040,0.460,-0.420
1960-06-01,42.600,10.700,93.300,3353.324,988.000,165.089,1271.000,69.480,9.400,3.320,...,-227.000,0.000,-0.900,-0.070,0.000,0.000,0.000,-0.190,-0.770,0.580
1960-07-01,44.400,11.000,93.300,3353.324,966.000,165.089,1247.000,69.480,8.400,3.230,...,-256.000,0.000,-2.800,-0.240,0.000,0.000,0.600,-0.500,-0.920,0.420
1960-08-01,43.700,10.800,97.200,3353.324,1013.000,165.089,1197.000,69.480,10.400,2.980,...,-350.000,0.000,-0.200,-0.520,0.000,0.000,0.300,-0.630,-1.110,0.480
1960-09-01,47.600,11.400,97.200,3353.324,1018.000,165.089,1344.000,69.480,10.400,2.600,...,-86.000,0.000,-0.100,-1.160,0.000,0.000,0.400,-0.880,-1.610,0.730
1960-10-01,45.400,11.000,97.200,3353.324,1027.000,165.089,1097.000,69.480,10.400,2.470,...,-443.000,0.000,-0.300,-1.510,0.000,0.000,0.100,-0.640,-1.800,1.160


In [190]:
model.feature_importances_

array([0.03304328, 0.02631926, 0.01798971, 0.0242312 , 0.01107705,
       0.02828376, 0.02695181, 0.03212652, 0.01693567, 0.        ,
       0.015511  , 0.02213848, 0.        , 0.02284955, 0.03221222,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.02938263, 0.        , 0.01647476, 0.02378056,
       0.        , 0.02535713, 0.        , 0.01763653, 0.02613422,
       0.02902528, 0.        , 0.02050433, 0.        , 0.01767201,
       0.01980311, 0.0194707 , 0.01486863, 0.02185273, 0.0292142 ,
       0.02762687, 0.        , 0.01551733, 0.        , 0.02696691,
       0.02298636, 0.00363962, 0.        , 0.        , 0.02232132,
       0.        , 0.        , 0.02149237, 0.0162576 , 0.01687958,
       0.        , 0.02901169, 0.02337376, 0.        , 0.        ,
       0.02400328, 0.        , 0.        , 0.        , 0.03066969,
       0.02978411, 0.02036081, 0.        , 0.01536524, 0.03289721],
      dtype=float32)

In [191]:
for combo in zip(X_train.columns,model.feature_importances_):
    print(combo)

('PMI', 0.033043284)
('YUNR', 0.026319258)
('CONS_SENT', 0.017989706)
('HOME_SALES', 0.024231201)
('PART_TIME', 0.011077045)
('CPPR_PRICE', 0.028283758)
('HOUS_STARTS', 0.026951812)
('CAP_UTIL', 0.032126516)
('PERS_SAVINGS', 0.016935669)
('INT_RATE', 0.0)
('US_NHOME_SALES', 0.0155110005)
('3YRT', 0.02213848)
('UNR', 0.0)
('10 Year Treasury Yield', 0.022849554)
('3 Month Treasury Yield (Bond Equivalent Basis)', 0.03221222)
('Spread', 0.0)
('HOME_SALES_PXY', 0.0)
('CPPR_PRICE_PXY', 0.0)
('HOUS_STARTS_PXY', 0.0)
('CAP_UTIL_PXY', 0.0)
('PERS_SAVINGS_PXY', 0.0)
('US_NHOME_SALES_PXY', 0.029382633)
('PMI_1m_shift', 0.0)
('YUNR_1m_shift', 0.016474761)
('CONS_SENT_1m_shift', 0.023780558)
('HOME_SALES_1m_shift', 0.0)
('PART_TIME_1m_shift', 0.02535713)
('CPPR_PRICE_1m_shift', 0.0)
('HOUS_STARTS_1m_shift', 0.017636528)
('CAP_UTIL_1m_shift', 0.026134217)
('PERS_SAVINGS_1m_shift', 0.029025275)
('INT_RATE_1m_shift', 0.0)
('US_NHOME_SALES_1m_shift', 0.02050433)
('3YRT_1m_shift', 0.0)
('UNR_1m_shift', 

In [192]:
importance = pd.DataFrame(zip(X_train.columns,model.feature_importances_), columns = ['Indicator','Importance'])

In [193]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(importance.sort_values(['Importance'], ascending = False))

                                            Indicator  Importance
0                                                 PMI       0.033
69                                   Spread_12m_shift       0.033
14     3 Month Treasury Yield (Bond Equivalent Basis)       0.032
7                                            CAP_UTIL       0.032
64                           US_NHOME_SALES_12m_shift       0.031
65                                     3YRT_12m_shift       0.030
21                                 US_NHOME_SALES_PXY       0.029
39                                      YUNR_3m_shift       0.029
30                              PERS_SAVINGS_1m_shift       0.029
56                                CONS_SENT_12m_shift       0.029
5                                          CPPR_PRICE       0.028
40                                 CONS_SENT_3m_shift       0.028
44                               HOUS_STARTS_3m_shift       0.027
6                                         HOUS_STARTS       0.027
1         

In [1282]:
X_train.head(100)

,PMI,YUNR,CONS_SENT,HOME_SALES,PART_TIME,CPPR_PRICE,HOUS_PERMS,HOUS_STARTS,CAP_UTIL,PERS_SAVINGS,EXPORTS,IMPORTS,TRADE_BALANCE,INT_RATE,US_M2,US_NHOME_SALES,PPI,CPI,3YRT,UNR,10 Year Treasury Yield,3 Month Treasury Yield (Bond Equivalent Basis),Spread,HOME_SALES_PXY,CPPR_PRICE_PXY,HOUS_PERMS_PXY,HOUS_STARTS_PXY,CAP_UTIL_PXY,PERS_SAVINGS_PXY,US_M2_PXY,US_NHOME_SALES_PXY,CPI_PXY,PMI_1m_shift,YUNR_1m_shift,CONS_SENT_1m_shift,HOME_SALES_1m_shift,PART_TIME_1m_shift,CPPR_PRICE_1m_shift,HOUS_PERMS_1m_shift,HOUS_STARTS_1m_shift,CAP_UTIL_1m_shift,PERS_SAVINGS_1m_shift,EXPORTS_1m_shift,IMPORTS_1m_shift,TRADE_BALANCE_1m_shift,INT_RATE_1m_shift,US_M2_1m_shift,US_NHOME_SALES_1m_shift,3YRT_1m_shift,UNR_1m_shift,10 Year Treasury Yield_1m_shift,3 Month Treasury Yield (Bond Equivalent Basis)_1m_shift,Spread_1m_shift,PMI_1m_shift_1m_shift,YUNR_1m_shift_1m_shift,CONS_SENT_1m_shift_1m_shift,HOME_SALES_1m_shift_1m_shift,PART_TIME_1m_shift_1m_shift,CPPR_PRICE_1m_shift_1m_shift,HOUS_PERMS_1m_shift_1m_shift,HOUS_STARTS_1m_shift_1m_shift,CAP_UTIL_1m_shift_1m_shift,PERS_SAVINGS_1m_shift_1m_shift,EXPORTS_1m_shift_1m_shift,PMI_3m_shift,YUNR_3m_shift,CONS_SENT_3m_shift,HOME_SALES_3m_shift,PART_TIME_3m_shift,CPPR_PRICE_3m_shift,HOUS_PERMS_3m_shift,HOUS_STARTS_3m_shift,CAP_UTIL_3m_shift,PERS_SAVINGS_3m_shift,EXPORTS_3m_shift,IMPORTS_3m_shift,TRADE_BALANCE_3m_shift,INT_RATE_3m_shift,US_M2_3m_shift,US_NHOME_SALES_3m_shift,3YRT_3m_shift,UNR_3m_shift,10 Year Treasury Yield_3m_shift,3 Month Treasury Yield (Bond Equivalent Basis)_3m_shift,Spread_3m_shift,PMI_1m_shift_3m_shift,YUNR_1m_shift_3m_shift,CONS_SENT_1m_shift_3m_shift,HOME_SALES_1m_shift_3m_shift,PART_TIME_1m_shift_3m_shift,CPPR_PRICE_1m_shift_3m_shift,HOUS_PERMS_1m_shift_3m_shift,HOUS_STARTS_1m_shift_3m_shift,CAP_UTIL_1m_shift_3m_shift,PERS_SAVINGS_1m_shift_3m_shift,EXPORTS_1m_shift_3m_shift,PMI_12m_shift,YUNR_12m_shift,CONS_SENT_12m_shift,HOME_SALES_12m_shift,PART_TIME_12m_shift,CPPR_PRICE_12m_shift,HOUS_PERMS_12m_shift,HOUS_STARTS_12m_shift,CAP_UTIL_12m_shift,PERS_SAVINGS_12m_shift,EXPORTS_12m_shift,IMPORTS_12m_shift,TRADE_BALANCE_12m_shift,INT_RATE_12m_shift,US_M2_12m_shift,US_NHOME_SALES_12m_shift,3YRT_12m_shift,UNR_12m_shift,10 Year Treasury Yield_12m_shift,3 Month Treasury Yield (Bond Equivalent Basis)_12m_shift,Spread_12m_shift,PMI_1m_shift_12m_shift,YUNR_1m_shift_12m_shift,CONS_SENT_1m_shift_12m_shift,HOME_SALES_1m_shift_12m_shift,PART_TIME_1m_shift_12m_shift,CPPR_PRICE_1m_shift_12m_shift,HOUS_PERMS_1m_shift_12m_shift,HOUS_STARTS_1m_shift_12m_shift,CAP_UTIL_1m_shift_12m_shift,PERS_SAVINGS_1m_shift_12m_shift,EXPORTS_1m_shift_12m_shift,CPI_1m_shift,PPI_1m_shift,CPI_3m_shift,PPI_3m_shift,CPI_12m_shift,PPI_12m_shift
1960-01-01,58.200,11.100,93.800,3353.324,1000.000,165.089,1331.222,1601.000,69.480,9.400,1328.000,1184.000,144.000,3.990,297.800,606.030,31.500,13.483,5.572,5.200,4.720,4.460,0.260,1,1,1,0,1,0,0,1,0,7.600,-0.200,0.000,0.000,7.000,0.000,0.000,185.000,0.000,0.700,-200.000,-227.000,27.000,0.000,0.700,0.000,0.000,-0.400,0.030,-0.140,0.170,6.700,-0.000,0.000,0.000,89.000,0.000,0.000,124.000,0.000,1.600,-296.000,9.900,-0.200,-1.500,0.000,113.000,0.000,0.000,61.000,0.000,-1.300,-69.000,-43.000,-26.000,0.010,1.100,0.000,0.000,-0.400,0.190,0.310,-0.120,14.400,-0.300,0.000,0.000,103.000,0.000,0.000,75.000,0.000,0.500,-240.000,-2.300,-1.000,3.000,0.000,-81.000,0.000,0.000,171.468,0.000,0.621,-21.000,93.000,-114.000,1.510,-3730.924,0.000,0.000,-0.900,0.700,1.580,-0.880,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.000,0.000,0.341,-0.631,1.730,-0.316
1960-02-01,61.500,10.900,93.800,3353.324,1015.000,165.089,1092.000,1460.000,69.480,10.100,1376.000,1292.000,84.000,3.970,298.200,606.030,31.600,13.437,5.572,5.200,4.490,4.060,0.430,1,1,0,0,1,0,0,1,0,3.300,-0.200,0.000,0.000,15.000,0.000,-239.222,-141.000,0.000,0.700,48.000,108.000,-60.000,-0.020,0.400,0.000,0.000,0.000,-0.230,-0.400,0.170,-4.300,0.000,0.000,0.000,8.000,0.000,-239.222,-326.000,0.000,-0.000,248.000,11.800,-0.600,0.000,0.000,-60.000,0.000,-23